In [422]:
from copy import deepcopy
import json
import os
from typing import List
import uuid
import re
import pandas as pd

In [423]:
def drop_duplicates(ls: List) -> List:
    return list(filter(None, set(ls)))

def flatten(ls: List) -> List:
    return [item for sublist in ls for item in sublist]

def remove(ls: List, to_remove: List) -> List:
    ls_copy = deepcopy(ls)
    for el in to_remove:
        if el in ls_copy:
            ls_copy.remove(el)
    return ls_copy

In [424]:
def load_perguntas():
    df = pd.read_excel("results/Perguntas.xlsx", sheet_name="finais")
    df = df.dropna(subset=["Resposta"])
    subs = {
        "Pergunta": "pergunta",
        "Resposta": "resposta",
        "Intenção": "intent",
        "Rótulos": "rótulos",
        "Modificador": "modificador",
        "Substantivo": "substantivo",
        "Recipiente": "recipiente",
        "Elocuções": "exemplos",
    }
    cols = list(subs.keys())
    df = df[cols]
    df = df.rename(columns=subs)
    df = df.fillna("")
    return df

In [425]:
def load_skill(skill_path):
    with open(skill_path, "r", encoding="utf-8") as f:
        skill = json.load(f)
    return skill

def save_skill(skill_path, skill):
    skill_root, skill_ext = os.path.splitext(skill_path)
    new_skill_path = f"{skill_root}2{skill_ext}"
    with open(new_skill_path, "w", encoding="utf-8") as f:
        json.dump(skill, f, ensure_ascii=False)

In [426]:
def get_intents(df):
    subset = ["intent", "pergunta", "exemplos"]
    records = df[subset].to_dict(orient="records")
    intents = [
        {
            "intent": record["intent"],
            "examples": get_examples(record),
            "description": "",
        }
        for record in records
    ]
    return intents

def get_examples(record) -> List:
    out = [{"text": record["pergunta"]}]
    if record["exemplos"]:
        out += [{"text": exemplo} for exemplo in record["exemplos"].split("--")]
    return out

In [427]:
def get_entities(df):
    subset = ["rótulos", "modificador", "substantivo", "recipiente"]
    entities = []
    for col in subset:
        records = df[col].drop_duplicates().to_list()
        records = [r.split("-") for r in records]
        records = flatten(records)
        records = drop_duplicates(records)

        values = [
            {"type": "synonyms", "value": record, "synonyms": []} for record in records
        ]
        entity = {"entity": col, "values": values, "fuzzy_match": True}
        entities.append(entity)
    return entities


In [428]:
def get_synonyms(a, b):
    """
    Pega sinônimos que foram adicionados a entidades através da interface do Watson
    e copia para as entidades geradas automaticamente da planilha.
    """
    ents_old = deepcopy(a)
    ents_new = deepcopy(b)
    for col in ["modificador", "rótulos", "substantivo", "recipiente"]:
        try:
            ent_old = next(ent for ent in ents_old if ent["entity"] == col)
            ent_new = next(ent for ent in ents_new if ent["entity"] == col)
        except StopIteration:
            continue
        for v_old in ent_old["values"]:
            for v_new in ent_new["values"]:
                if v_new["value"] == v_old["value"]:
                    v_new["synonyms"] = v_old["synonyms"]
    return ents_new

In [429]:
def get_contextos(rotulos):
    rotulos_nao_contextuais = [
        "fauna",
        "flora",
        "extra",
        "física",
        "símbolo",
        "turismo",
        "engenharia",
        "saúde",
        "geologia",
    ]
    contextos = [contexto for contexto in rotulos if all([rot not in contexto for rot in rotulos_nao_contextuais])]
    return contextos

def get_titulo(js):
    modificador = js["modificador"]
    substantivo = js["substantivo"].replace("-", " ")
    recipiente = js["recipiente"]
    
    contextos = get_contextos(js["rótulos"].split("-"))
    contextos = list(filter(lambda x: x.count("-") == 0, contextos))
    titulo = f"{'/'.join(map(str.title, contextos))}: " if contextos else ""
    titulo += modificador + " "
    
    if modificador in ["efeito"]:
        titulo += f" de {substantivo} em {recipiente}"
    elif modificador in ["maiores", "menores"]:
        subst = {"produção": "produtores"}
        if recipiente in subst:
            recipiente = subst[recipiente]
        substantivo = substantivo + "s" if substantivo[-1] != "s" else substantivo
        titulo += f"{substantivo} {recipiente}"
    elif modificador in ["diferença"]:
        recipiente = recipiente or contextos[0]
        titulo += f"entre {substantivo} e {recipiente}"
    elif modificador in ["existe", "quantidade"]:
        recipiente = recipiente or "no Brasil"
        if substantivo:
            titulo += substantivo + " "
        titulo += f"{recipiente}"
    elif modificador in ["listar"]:
        subst = {"extinção": "em extinção", "aaz": "na Amazônia Azul"}
        if recipiente in subst:
            recipiente = subst[recipiente]
        titulo += f"{substantivo} {recipiente}"
    else:
        if substantivo:
            titulo += substantivo + " "
        if recipiente:
            titulo += recipiente + " "
    titulo = titulo.strip()
    return titulo

def get_condition_string(js):
    modificador = js["modificador"]
    substantivo = js["substantivo"]
    recipiente = js["recipiente"]
    rotulos = js["rótulos"].split("-") + [js["rótulos"]]
    rotulos = drop_duplicates(rotulos)
    contextos = get_contextos(rotulos)

    if contextos:
        conds_adicionais = [
            [
                " && ".join(
                    [
                        f"@modificador:{modificador}",
                        (f"@substantivo:{substantivo}" if substantivo else ""),
                        (f"@recipiente:{recipiente}" if recipiente else ""),
                        f'$contexto=="{contexto}"',
                    ]
                ),
                " && ".join(
                    [
                        f"@modificador:{modificador}",
                        (f"@substantivo:{substantivo}" if substantivo else ""),
                        (f"@recipiente:{recipiente}" if recipiente else ""),
                        f"@rótulos:{contexto}",
                    ]
                ),
            ]
            for contexto in contextos
        ]
    else:
        conds_adicionais = [
            [
                " && ".join(
                    [
                        f"@modificador:{modificador}",
                        (f"@substantivo=={substantivo}" if substantivo else ""),
                        (f"@recipiente=={recipiente}" if recipiente else ""),
                    ]
                )
            ]
        ]

    conds_adicionais = list(
        map(
            lambda x: map(lambda y: re.sub(r"(&  &)| && $", "", y), x), conds_adicionais
        )
    )
    conds = [f"#{js['intent']}"] + flatten(conds_adicionais)
    cond_str = " || ".join(conds)
    return cond_str
    

def get_dialog_nodes(df):
    records = df.to_dict(orient="records")
    dialog_nodes = [
        {
            "type": "standard",
            "title": get_titulo(record),
            "output": {
                "generic": [
                    {
                        "values": [{"text": record["resposta"]}],
                        "response_type": "text",
                        "selection_policy": "sequential",
                    }
                ]
            },
            "context": {"contexto": record["rótulos"]},
            "conditions": get_condition_string(record),
            "dialog_node": f"node_{uuid.uuid4().hex[:16]}"
        }
        for record in records
    ]
    
    # atribuir previous siblings
    for i in range(len(dialog_nodes) - 1):
        prev, node = dialog_nodes[i], dialog_nodes[i + 1]
        node["previous_sibling"] = prev["dialog_node"]
        
    return dialog_nodes

In [430]:
def get_skill(skill, **kwargs):
    new_skill = deepcopy(skill)
    for k, v in kwargs.items():
        new_skill[k] = v
    return new_skill

In [431]:
df = load_perguntas()
df[~(df["exemplos"]=="")]

,pergunta,resposta,intent,rótulos,modificador,substantivo,recipiente,exemplos
56,Por que tartarugas estão em extinção?,"Além dos predadores naturais, a principal amea...",tartaruga--motivo-extinção,tartaruga,motivo,,extinção,Quais são as ameaças às tartarugas?
172,O que é Chordata?,"Chordata, um dos filos do reino Animalia, é ca...",fauna--definir-cordado,fauna,definir,cordado,,O que são cordados?
174,O que é uma cnida?,Cnida (ou nematocisto) é uma cápsula que conté...,fauna--definir-cnida,fauna,definir,cnida,,O que é um nematocisto?


In [432]:
skill_path = "results/skill-Amazônia-Azul.json"
skill = load_skill(skill_path)

In [433]:
new_intents = get_intents(df)
new_intents

[{'intent': 'coral--definir',
  'examples': [{'text': 'O que é um coral?'}],
  'description': ''},
 {'intent': 'coral--composição',
  'examples': [{'text': 'Do que é feito um coral?'}],
  'description': ''},
 {'intent': 'petróleo--maiores-reserva',
  'examples': [{'text': 'Quais são as maiores reservas de petróleo no Brasil?'}],
  'description': ''},
 {'intent': 'gás--maiores-estado-produção',
  'examples': [{'text': 'Que estados produzem mais petróleo?'}],
  'description': ''},
 {'intent': 'petróleo--maiores-estado-produção',
  'examples': [{'text': 'Que estados produzem mais gás?'}],
  'description': ''},
 {'intent': 'petróleo--quantidade',
  'examples': [{'text': 'Quanto petróleo há no Brasil?'}],
  'description': ''},
 {'intent': 'gás--quantidade-consumo',
  'examples': [{'text': 'Qual é o consumo de gás brasileiro?'}],
  'description': ''},
 {'intent': 'petróleo--localização',
  'examples': [{'text': 'Onde o petróleo é encontrado?'}],
  'description': ''},
 {'intent': 'petróleo',


In [434]:
new_entities = get_entities(df)
new_entities

[{'entity': 'rótulos',
  'values': [{'type': 'synonyms', 'value': 'gás', 'synonyms': []},
   {'type': 'synonyms', 'value': 'petróleo', 'synonyms': []},
   {'type': 'synonyms', 'value': 'extra', 'synonyms': []},
   {'type': 'synonyms', 'value': 'tartaruga', 'synonyms': []},
   {'type': 'synonyms', 'value': 'turismo', 'synonyms': []},
   {'type': 'synonyms', 'value': 'flora', 'synonyms': []},
   {'type': 'synonyms', 'value': 'fauna', 'synonyms': []},
   {'type': 'synonyms', 'value': 'coral', 'synonyms': []}],
  'fuzzy_match': True},
 {'entity': 'modificador',
  'values': [{'type': 'synonyms', 'value': 'listar', 'synonyms': []},
   {'type': 'synonyms', 'value': 'localização', 'synonyms': []},
   {'type': 'synonyms', 'value': 'definir', 'synonyms': []},
   {'type': 'synonyms', 'value': 'diferença', 'synonyms': []},
   {'type': 'synonyms', 'value': 'quantidade', 'synonyms': []},
   {'type': 'synonyms', 'value': 'composição', 'synonyms': []},
   {'type': 'synonyms', 'value': 'maiores', 'syno

In [435]:
new_entities = get_synonyms(skill["entities"], new_entities)
new_entities

[{'entity': 'rótulos',
  'values': [{'type': 'synonyms', 'value': 'gás', 'synonyms': []},
   {'type': 'synonyms', 'value': 'petróleo', 'synonyms': []},
   {'type': 'synonyms', 'value': 'extra', 'synonyms': []},
   {'type': 'synonyms', 'value': 'tartaruga', 'synonyms': []},
   {'type': 'synonyms', 'value': 'turismo', 'synonyms': []},
   {'type': 'synonyms', 'value': 'flora', 'synonyms': []},
   {'type': 'synonyms', 'value': 'fauna', 'synonyms': []},
   {'type': 'synonyms', 'value': 'coral', 'synonyms': []}],
  'fuzzy_match': True},
 {'entity': 'modificador',
  'values': [{'type': 'synonyms', 'value': 'listar', 'synonyms': []},
   {'type': 'synonyms', 'value': 'localização', 'synonyms': []},
   {'type': 'synonyms', 'value': 'definir', 'synonyms': ['defina']},
   {'type': 'synonyms', 'value': 'diferença', 'synonyms': []},
   {'type': 'synonyms', 'value': 'quantidade', 'synonyms': []},
   {'type': 'synonyms', 'value': 'composição', 'synonyms': []},
   {'type': 'synonyms', 'value': 'maiores

In [436]:
new_dialog_nodes = get_dialog_nodes(df)
new_dialog_nodes

[{'type': 'standard',
  'title': 'Coral: definir',
  'output': {'generic': [{'values': [{'text': 'Corais são cnidários que possuem um exoesqueleto calcário ou de matéria orgânica, o que os diferencia de anêmonas.'}],
     'response_type': 'text',
     'selection_policy': 'sequential'}]},
  'context': {'contexto': 'coral'},
  'conditions': '#coral--definir || @modificador:definir && $contexto=="coral" || @modificador:definir && @rótulos:coral',
  'dialog_node': 'node_9f3bbdd49bcf4f17'},
 {'type': 'standard',
  'title': 'Coral: composição',
  'output': {'generic': [{'values': [{'text': 'Um coral pode ser constituído de um ou mais pólipos adultos. É comum que possuam um exoesqueleto calcário, ou seja, sejam bem duros por fora.'}],
     'response_type': 'text',
     'selection_policy': 'sequential'}]},
  'context': {'contexto': 'coral'},
  'conditions': '#coral--composição || @modificador:composição && $contexto=="coral" || @modificador:composição && @rótulos:coral',
  'dialog_node': 'node

In [437]:
new_skill = get_skill(skill, intents=new_intents, entities=new_entities, dialog_nodes=new_dialog_nodes)

In [438]:
save_skill(skill_path, new_skill)